In [1]:
import pandas as pd
from collections import Counter
from datetime import datetime
import re
from utils import *
import altair as alt
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
# import these modules
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.sentiment import SentimentIntensityAnalyzer
from LeXmo import LeXmo

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vcaminic\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data = pd.read_csv('tripadvisor.csv')

In [3]:
data, emojis, tokens_df, bigr_with_sw, trigr_with_sw, tokens, corpus, words = dfCleaner(
    data)

c:\Users\vcaminic\Desktop\00_CODE\Studies\ScripAdvisor\utils.py:128: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  emo_df = data['review_body'].apply(lambda x: LeXmo.LeXmo(x)).apply(pd.Series).drop(


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from transformers import pipeline

# Load Aspect-Based Sentiment Analysis model
absa_tokenizer = AutoTokenizer.from_pretrained(
    "yangheng/deberta-v3-base-absa-v1.1")
absa_model = AutoModelForSequenceClassification \
    .from_pretrained("yangheng/deberta-v3-base-absa-v1.1")

# Load a traditional Sentiment Analysis model
sentiment_model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_model = pipeline("sentiment-analysis", model=sentiment_model_path,
                           tokenizer=sentiment_model_path)

c:\Users\vcaminic\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [5]:
sentence = 'food is bad'
print(f"Sentence: {sentence}")
print()

# ABSA of "food"
aspect = "food"
inputs = absa_tokenizer(
    f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")
outputs = absa_model(**inputs)
probs = F.softmax(outputs.logits, dim=1)
probs = probs.detach().numpy()[0]
print(f"Sentiment of aspect '{aspect}' is:")
for prob, label in zip(probs, ["negative", "neutral", "positive"]):
    print(f"Label {label}: {prob}")
print()
# Sentiment of aspect 'food' is:
# Label negative: 0.0009989114478230476
# Label neutral: 0.001823813421651721
# Label positive: 0.997177243232727

# # ABSA of "service"
# aspect = "service"
# inputs = absa_tokenizer(
#     f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")
# outputs = absa_model(**inputs)
# probs = F.softmax(outputs.logits, dim=1)
# probs = probs.detach().numpy()[0]
# print(f"Sentiment of aspect '{aspect}' is:")
# for prob, label in zip(probs, ["negative", "neutral", "positive"]):
#     print(f"Label {label}: {prob}")
# print()
# Sentiment of aspect 'service' is:
# Label negative: 0.9946129322052002
# Label neutral: 0.002369985682889819
# Label positive: 0.003017079783603549

# Overall sentiment of the sentence
sentiment = sentiment_model([sentence])[0]
print(
    f"Overall sentiment: {sentiment['label']} with score {sentiment['score']}")
# Overall sentiment: Negative with score 0.7706006765365601

Sentence: food is bad

Sentiment of aspect 'food' is:
Label negative: 0.9921167492866516
Label neutral: 0.0059877787716686726
Label positive: 0.0018954770639538765

Overall sentiment: negative with score 0.9428786039352417


In [28]:
sentence = 'mi fai completamente schifo'

print(sentiment_model([sentence])[0]['label'])

negative


In [59]:
def absa_predictor(sentence, word):

    sentence = sentence
    aspect = word
    inputs = absa_tokenizer(
        f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")
    outputs = absa_model(**inputs)
    probs = F.softmax(outputs.logits, dim=1)
    probs = probs.detach().numpy()[0]
    labels = dict(zip(["negative", "neutral", "positive"], probs))
    result = max(labels, key=labels.get)
    # print(sentence)
    print(
        f'Sentiment for {aspect} is {result} with a prob of: {max(labels.values())}')

    return

In [63]:
absa_predictor('Perfectly located for accessing the old town and wandering its narrow streets. \
    Sunday has a small flea market which is worth a browse when enjoying an expresso. \
        Hotel is average, room ok, we had an attic room which had noises from the pigeons on the roof.',
               'position')

absa_predictor('Perfectly located for accessing the old town and wandering its narrow streets. \
    Sunday has a small flea market which is worth a browse when enjoying an expresso. \
        Hotel is average, room ok, we had an attic room which had noises from the pigeons on the roof.',
               'room')

Sentiment for position is positive with a prob of: 0.9670966267585754
Sentiment for room is neutral with a prob of: 0.5143488645553589


In [39]:
max(result.values())

0.9891127

In [35]:
result

{'negative': 0.9891127, 'neutral': 0.004781139, 'positive': 0.006106108}

In [58]:
data.loc[4, 'review_body']

'we did a corporate event with 125 pax at the royal plaza delhi on 24th june 23 and are 100 satisfied with the quality of services banquet manager was extremely helpful and even handled some last minute requirements from our side food and hospitality was up to the mark really recommend the royal plaza delhi to all '

In [32]:
sentence = 'questo telefpmp fa schifo, ma il servizio è veramente ottimo'

print(absa_predictor(sentence, 'telefono'))


print(sentiment_model([sentence])[0]['label'])

negative
negative
